In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os

os.chdir("/content/drive/MyDrive/Spelling Correction")
os.getcwd()

'/content/drive/MyDrive/Spelling Correction'

In [4]:
!pip install symspellpy

In [6]:
import argparse
from symspellpy.symspellpy import SymSpell

import pandas as pd
from tensorflow.keras.preprocessing.text import text_to_word_sequence

import warnings
warnings.filterwarnings("ignore")

In [7]:
df = pd.read_csv("sample.csv", encoding = "CP949")
sentence = list(df.iloc[:,0])
organ_list=sentence
organ_list

['enterococcus avium - (group d)',
 'escherichia coli',
 'haemophilus parpinfluenzae',
 'klebsiella pneumoniae',
 'leclercia adecarboxylate',
 'staphylococcus aureus']

In [8]:
stop_words=["ss","spp","ssp","mrsa","mssa","group"]
print("Stop words:" + str(stop_words))

Stop words:['ss', 'spp', 'ssp', 'mrsa', 'mssa', 'group']


### 1. Extraction of data using regular expression

In [47]:
# Word
organ_word=[]
for _ in range(len(organ_list)):
    
    temp_words = text_to_word_sequence(organ_list[_]) #Tokenizer
    
    for _ in temp_words:
        if _ not in stop_words:
            if len(_)>3:
                organ_word.append(_)
                
# Sentence
organ_sentence=[]
for _ in range(len(organ_list)):
    
    temp_token=[]
    temp_words = text_to_word_sequence(organ_list[_]) #Tokenizer
    
    for _ in temp_words:
        if _ not in stop_words:
            if len(_)>3:
                temp_token.append(_)
                
    organ_sentence.append(temp_token)

for _ in range(len(organ_sentence)):
    organ_sentence[_] =  " ".join(organ_sentence[_])

In [15]:
organ_corpus = sorted(list(set(organ_word)))
len(organ_corpus)

12

In [50]:
organ_sentence

['enterococcus avium',
 'escherichia coli',
 'haemophilus parpinfluenzae',
 'klebsiella pneumoniae',
 'leclercia adecarboxylate',
 'staphylococcus aureus']

### 2. Misspelling Detection

In [51]:
dict_corpus=list(pd.read_csv("data/misspelling_detection.txt", header=None)[0])
misspell=sorted(list(set(organ_word)))
for _ in dict_corpus:
    if _ in misspell: 
        misspell.remove(_)

misspell

['adecarboxylate', 'parpinfluenzae']

### 3. SymSpell

In [52]:
sym_spell = SymSpell(max_dictionary_edit_distance=3)
sym_spell.create_dictionary("data/SNOMED_dictionary.txt", encoding="UTF-8")
sym_spell.load_dictionary("data/frequency_dictionary_en_82_765.txt", 0, 1)

for mis in misspell:
    suggestions = sym_spell.lookup_compound(mis, max_edit_distance=1)
    for suggestion in suggestions:
        print("{}, {}, {}".format(suggestion.term, suggestion.distance, suggestion.count))

adecarboxylata, 1, 0
parainfluenzae, 1, 9


In [53]:
%%time
organ_rev=[]
for mis in organ_corpus:
    suggestions = sym_spell.lookup_compound(mis, max_edit_distance=3)
    for suggestion in suggestions:
        organ_rev.append(suggestion.term)
        
len(sorted(list(set(organ_rev))))

CPU times: user 94.8 ms, sys: 1.99 ms, total: 96.8 ms
Wall time: 96.9 ms


In [ ]:
ss

In [54]:
organ_rev

['adecarboxylata',
 'aureus',
 'avium',
 'coli',
 'enterococcus',
 'escherichia',
 'haemophilus',
 'klebsiella',
 'leclercia',
 'parainfluenzae',
 'pneumoniae',
 'staphylococcus']

In [ ]:
pd.Series(organ_rev).to_csv("rev.csv", index=False, encoding="CP949")